In [1]:
import torch
import pandas as pd
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
dataset = pd.read_csv("first_dataset_bm25.csv")
dataset["bm25_k=1_pages"] = dataset["bm25_k=1_pages"].str.slice(1, -1)
dataset["Page"] = dataset["Page"].astype(int)
dataset["bm25_k=1_pages"] = dataset["bm25_k=1_pages"].astype(int)

In [4]:
test = dataset.iloc[(dataset["Page"] == dataset["bm25_k=1_pages"]).values]

In [8]:
print(len(test), len(dataset))

9 21


In [9]:
dataset['BERT Answer'] = ""

for index, row in dataset.iterrows():
    question = row["Question"]
    paragraph = row["Paragraph"]

    encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, truncation=True, padding="max_length", max_length=512)

    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    res = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    start_scores, end_scores = res['start_logits'], res['end_logits']

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = ' '.join(tokens[start_index:end_index+1])

    corrected_answer = ''

    for word in answer.split():
        
        #If it's a subword token
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    dataset.at[index, 'BERT Answer'] = corrected_answer

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [13]:
selected_columns = dataset.iloc[:, [-1, -3]]
selected_columns.to_csv('bert_test_21samples.csv', index=False)

In [36]:
test.to_csv('test.csv', index=False)
